###  Assignment 7.1 a

In [3]:
import pandas as pd

# read data from parquet file
df = pd.read_parquet('C:/Users/chris/DSC650-T301/DSC650-T301/Assignment03/Results/routes.parquet')
df['airline_name'] = df['airline'].apply(lambda x: x['name'])
df['airline_id'] = df['airline'].apply(lambda x: x['airline_id'])
df['airline_name_iata'] = df['airline'].apply(lambda x: x['iata'])
df['src_airport'] = df['src_airport'].fillna({})
df['src_airport_name'] = df['src_airport'].apply(lambda x: x.get('name') if bool(x) and 'name' in x else None)
df['src_airport_city'] = df['src_airport'].apply(lambda x: x.get('city')if bool(x) and 'city' in x else None)
df['src_airport_iata'] = df['src_airport'].apply(lambda x: x.get('iata') if bool(x) and 'iata' in x else None)
df['dest_airport_name'] = df['dst_airport'].apply(lambda x: x.get('name') if bool(x) and 'name' in x else None)
df['dest_airport_city'] = df['dst_airport'].apply(lambda x: x.get('city')if bool(x) and 'city' in x else None)
df['dest_airport_iata'] = df['dst_airport'].apply(lambda x: x.get('iata')if bool(x) and 'iata' in x else None)
df['src_airport_latitude'] = df['src_airport'].apply(lambda x: x.get('latitude') if bool(x) and 'latitude' in x else None)
df['src_airport_longitude'] = df['src_airport'].apply(lambda x: x.get('longitude') if bool(x) and 'longitude' in x else None)
df['key'] = df['src_airport_iata'] +  df['dest_airport_iata'] + df['airline_name_iata']
df = df.drop(['airline', 'src_airport', 'dst_airport','codeshare','stops','equipment'], axis=1)
df.head()


,airline_name,airline_id,airline_name_iata,src_airport_name,src_airport_city,src_airport_iata,dest_airport_name,dest_airport_city,dest_airport_iata,src_airport_latitude,src_airport_longitude,key
0,Aerocondor,410,2B,Sochi International Airport,Sochi,AER,Kazan International Airport,Kazan,KZN,43.449902,39.956600,AERKZN2B
1,Aerocondor,410,2B,Astrakhan Airport,Astrakhan,ASF,Kazan International Airport,Kazan,KZN,46.283298,48.006302,ASFKZN2B
2,Aerocondor,410,2B,Astrakhan Airport,Astrakhan,ASF,Mineralnyye Vody Airport,Mineralnye Vody,MRV,46.283298,48.006302,ASFMRV2B
3,Aerocondor,410,2B,Chelyabinsk Balandino Airport,Chelyabinsk,CEK,Kazan International Airport,Kazan,KZN,55.305801,61.503300,CEKKZN2B
4,Aerocondor,410,2B,Chelyabinsk Balandino Airport,Chelyabinsk,CEK,Tolmachevo Airport,Novosibirsk,OVB,55.305801,61.503300,CEKOVB2B


In [3]:
import os

# Define the partitions
partitions = (
    ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
    ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
    ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
    ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
)

partitions_keys = (
        'A', 'B', 'C-D', 'E-F',
        'G-H', 'I-J', 'K-L', 'M',
        'N', 'O-P', 'Q-R', 'S-T',
        'U', 'V', 'W-X', 'Y-Z'
    )
parts_k_v = dict(zip(partitions_keys, partitions))
print(parts_k_v)

{'A': ('A', 'A'), 'B': ('B', 'B'), 'C-D': ('C', 'D'), 'E-F': ('E', 'F'), 'G-H': ('G', 'H'), 'I-J': ('I', 'J'), 'K-L': ('K', 'L'), 'M': ('M', 'M'), 'N': ('N', 'N'), 'O-P': ('O', 'P'), 'Q-R': ('Q', 'R'), 'S-T': ('S', 'T'), 'U': ('U', 'U'), 'V': ('V', 'V'), 'W-X': ('W', 'X'), 'Y-Z': ('Y', 'Z')}


In [4]:
df['partition_value'] = df['key'].str[:1]
def get_key(val):
    for key, value in parts_k_v.items():
        if val in value:
            return key
    return "0"
df['kv_key'] = df.apply(lambda x: get_key(x.partition_value), axis=1)

In [5]:
# Save partitioned dataset to kv directory
df.to_parquet('results/kv', partition_cols=['kv_key'])

In [6]:
df.head()

,airline_name,airline_id,airline_name_iata,src_airport_name,src_airport_city,src_airport_iata,dest_airport_name,dest_airport_city,dest_airport_iata,src_airport_latitude,src_airport_longitude,key,partition_value,kv_key
0,Aerocondor,410,2B,Sochi International Airport,Sochi,AER,Kazan International Airport,Kazan,KZN,43.449902,39.956600,AERKZN2B,A,A
1,Aerocondor,410,2B,Astrakhan Airport,Astrakhan,ASF,Kazan International Airport,Kazan,KZN,46.283298,48.006302,ASFKZN2B,A,A
2,Aerocondor,410,2B,Astrakhan Airport,Astrakhan,ASF,Mineralnyye Vody Airport,Mineralnye Vody,MRV,46.283298,48.006302,ASFMRV2B,A,A
3,Aerocondor,410,2B,Chelyabinsk Balandino Airport,Chelyabinsk,CEK,Kazan International Airport,Kazan,KZN,55.305801,61.503300,CEKKZN2B,C,C-D
4,Aerocondor,410,2B,Chelyabinsk Balandino Airport,Chelyabinsk,CEK,Tolmachevo Airport,Novosibirsk,OVB,55.305801,61.503300,CEKOVB2B,C,C-D


###  Assignment 7.1 b

In [7]:
import hashlib
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

# Define the hash function
def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()


# Create the key column
df['key'] = df['src_airport_iata'] + df['dest_airport_iata'] + df['airline_name_iata']

# Create the hashed column
df['hashed'] = df['key'].apply(hash_key)

# Create the hash partition column
df['hash_key'] = df['hashed'].str[:1]

# Convert the data frame to a PyArrow table
pa_table = pa.Table.from_pandas(df)


# Write the PyArrow table to a Parquet dataset partitioned by hash_key
pq.write_to_dataset(pa_table, root_path='results/hash', partition_cols=['hash_key'])


In [8]:
df.head()

,airline_name,airline_id,airline_name_iata,src_airport_name,src_airport_city,src_airport_iata,dest_airport_name,dest_airport_city,dest_airport_iata,src_airport_latitude,src_airport_longitude,key,partition_value,kv_key,hashed,hash_key
0,Aerocondor,410,2B,Sochi International Airport,Sochi,AER,Kazan International Airport,Kazan,KZN,43.449902,39.956600,AERKZN2B,A,A,652cdec02010381f175efe499e070c8cbaac1522bac59a...,6
1,Aerocondor,410,2B,Astrakhan Airport,Astrakhan,ASF,Kazan International Airport,Kazan,KZN,46.283298,48.006302,ASFKZN2B,A,A,9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...,9
2,Aerocondor,410,2B,Astrakhan Airport,Astrakhan,ASF,Mineralnyye Vody Airport,Mineralnye Vody,MRV,46.283298,48.006302,ASFMRV2B,A,A,161143856af25bd4475f62c80c19f68936a139f653c1d3...,1
3,Aerocondor,410,2B,Chelyabinsk Balandino Airport,Chelyabinsk,CEK,Kazan International Airport,Kazan,KZN,55.305801,61.503300,CEKKZN2B,C,C-D,39aa99e6ae2757341bede9584473906ef1089e30820c90...,3
4,Aerocondor,410,2B,Chelyabinsk Balandino Airport,Chelyabinsk,CEK,Tolmachevo Airport,Novosibirsk,OVB,55.305801,61.503300,CEKOVB2B,C,C-D,143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...,1


###  Assignment 7.1 c

In [9]:
from math import radians, cos, sin, asin, sqrt
import pyarrow as pa
import pyarrow.parquet as pq

def haversine(lat1, lon1, lat2, lon2):
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# Define the locations of the data centers
data_centers = {
    'west': (45.5945645, -121.1786823),
    'central': (41.1544433, -96.0422378),
    'east': (39.08344, -77.6497145)
}

# Calculate the distance between each source airport and the data centers
for index, row in df.iterrows():
    lat1, lon1 = row['src_airport_latitude'], row['src_airport_longitude']
    distances = {}
    for location, coords in data_centers.items():
        lat2, lon2 = coords
        distance = haversine(lat1, lon1, lat2, lon2)
        distances[location] = distance
    closest_location = min(distances, key=distances.get)
    folder_name = f"location={closest_location}"
    
    # Select the rows corresponding to the current route
    route_df = df.loc[df['key'] == row['key']]

    # Convert the data frame to a PyArrow table
    route_table = pa.Table.from_pandas(route_df)

    # Write the route to the appropriate folder
    pq.write_table(route_table, "results/geo")





###  Assignment 7.1 d

In [10]:

df.head()

,airline_name,airline_id,airline_name_iata,src_airport_name,src_airport_city,src_airport_iata,dest_airport_name,dest_airport_city,dest_airport_iata,src_airport_latitude,src_airport_longitude,key,partition_value,kv_key,hashed,hash_key
0,Aerocondor,410,2B,Sochi International Airport,Sochi,AER,Kazan International Airport,Kazan,KZN,43.449902,39.956600,AERKZN2B,A,A,652cdec02010381f175efe499e070c8cbaac1522bac59a...,6
1,Aerocondor,410,2B,Astrakhan Airport,Astrakhan,ASF,Kazan International Airport,Kazan,KZN,46.283298,48.006302,ASFKZN2B,A,A,9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...,9
2,Aerocondor,410,2B,Astrakhan Airport,Astrakhan,ASF,Mineralnyye Vody Airport,Mineralnye Vody,MRV,46.283298,48.006302,ASFMRV2B,A,A,161143856af25bd4475f62c80c19f68936a139f653c1d3...,1
3,Aerocondor,410,2B,Chelyabinsk Balandino Airport,Chelyabinsk,CEK,Kazan International Airport,Kazan,KZN,55.305801,61.503300,CEKKZN2B,C,C-D,39aa99e6ae2757341bede9584473906ef1089e30820c90...,3
4,Aerocondor,410,2B,Chelyabinsk Balandino Airport,Chelyabinsk,CEK,Tolmachevo Airport,Novosibirsk,OVB,55.305801,61.503300,CEKOVB2B,C,C-D,143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...,1


In [4]:
def balance_partitions (keys, num_partitions):
    uniquevals = sorted(set(keys))
    num_uniq_vals = len(uniquevals)
    partition_counts = (num_uniq_vals / num_partitions)+1
    partitions  = []
    currow = 1
    partnum = 1
    for i in range(num_uniq_vals):
        curkeyval ={}
        if currow <= partition_counts:
            curkeyval[uniquevals[i]] = partnum
            currow = currow + 1
        else:
            currow = 1
            partnum = partnum + 1
            curkeyval[uniquevals[i]] = partnum
            currow = currow + 1
        partitions .append(curkeyval)
    return partitions


keys = df['airline_id']
num_partitions = 3
partitions  = balance_partitions(keys, num_partitions)
print(type(partitions ))
print(partitions )

<class 'list'>
[{-1: 1}, {10: 1}, {21: 1}, {24: 1}, {28: 1}, {29: 1}, {32: 1}, {35: 1}, {42: 1}, {43: 1}, {55: 1}, {68: 1}, {83: 1}, {90: 1}, {96: 1}, {106: 1}, {109: 1}, {116: 1}, {125: 1}, {130: 1}, {132: 1}, {137: 1}, {139: 1}, {146: 1}, {153: 1}, {179: 1}, {197: 1}, {214: 1}, {218: 1}, {220: 1}, {221: 1}, {225: 1}, {231: 1}, {240: 1}, {241: 1}, {242: 1}, {246: 1}, {312: 1}, {316: 1}, {319: 1}, {321: 1}, {324: 1}, {328: 1}, {329: 1}, {330: 1}, {333: 1}, {336: 1}, {338: 1}, {341: 1}, {345: 1}, {386: 1}, {397: 1}, {410: 1}, {412: 1}, {426: 1}, {439: 1}, {442: 1}, {462: 1}, {470: 1}, {476: 1}, {477: 1}, {491: 1}, {502: 1}, {503: 1}, {508: 1}, {515: 1}, {524: 1}, {543: 1}, {563: 1}, {567: 1}, {569: 1}, {576: 1}, {595: 1}, {596: 1}, {603: 1}, {608: 1}, {622: 1}, {641: 1}, {683: 1}, {690: 1}, {692: 1}, {751: 1}, {753: 1}, {794: 1}, {807: 1}, {837: 1}, {879: 1}, {881: 1}, {882: 1}, {896: 1}, {897: 1}, {921: 1}, {970: 1}, {995: 1}, {998: 1}, {1006: 1}, {1008: 1}, {1034: 1}, {1048: 1}, {1057